In [1]:
import numpy as np
import pymc as pm
import pandas as pd
import arviz as az
import matplotlib
import matplotlib.pyplot as plt
import os
import math
import pickle
import ast
import xarray as xr
from scipy.stats import binom

In [2]:
#load raw data
with open("..\\..\\Sirius_Data.pkl", "rb") as f:
    DataDict = pickle.load(f)  

Format Experimental Data

In [3]:
Raw_DataFrame = DataDict['data']
stims = np.array(Raw_DataFrame['stim_amp'])

#normalize stim amps
x_old = np.unique(stims)
x_mu = np.mean(x_old)
x_sig = np.std(x_old)
stims_normed = ((stims-x_mu)/x_sig)

#construct covarate matrix
cov_mat = np.vstack([np.full_like(stims_normed, 1.0), stims_normed]).T

#hand/distraction group indices
grp_idx = np.array(Raw_DataFrame['group_idx'])

#binary responses
resp = np.array(Raw_DataFrame['response'])

#construct dictionary
ReadyData_Sirius_A = {'cov_mat':cov_mat,'grp_idx':grp_idx,'resp': resp}

In [4]:
#save processed data
with open("ReadyData_Sirius_A.pkl","wb") as f:
    pickle.dump(ReadyData_Sirius_A, f)

Generate Synthetic Dataset

In [5]:
#fix parameters for each group
gam_h_fix =  np.array([0.03, 0.06, 0.06, 0.03])
gam_l_fix =  np.array([0.005, 0.01, 0.005, 0.01])
beta0_fix = np.array([-1, 1, -1, 1])
beta1_fix = np.array([3, 3, 6, 6])
params_fixed = [gam_h_fix, gam_l_fix, beta0_fix, beta1_fix]

In [6]:
#generates synthetic data from same covariates as experimental
def synth_generator_A(params_fixed, cov_mat, grp_idx):
    [gam_h_fix, gam_l_fix, beta0_fix, beta1_fix] = params_fixed
    beta_mat = np.vstack([beta0_fix, beta1_fix])
    log_arg = (cov_mat @ beta_mat)
    psis_mat = gam_h_fix + (1-gam_h_fix-gam_l_fix)/( 1 + np.exp( - log_arg ))
    psis = psis_mat[np.arange(len(grp_idx)), grp_idx]
    synth_resp = binom.rvs(n=1, p=psis)
    return synth_resp

In [7]:
synth_resp = synth_generator_A(params_fixed, cov_mat, grp_idx)

In [8]:
#construct dictionary
ReadyData_Synth_A = {'cov_mat': cov_mat,
                     'grp_idx': grp_idx,
                     'resp': synth_resp, 
                     'params_fixed': params_fixed}

In [9]:
#save processed data
with open("ReadyData_Synth_A.pkl","wb") as f:
    pickle.dump(ReadyData_Synth_A, f)